In [2]:
import pandas as pd
import numpy as np


In [3]:
data=pd.read_csv('../data/Silver/annotation/sample_data_with_annotation.csv')
data.head(5)

,type,content,aspect,title,TH1_aspect,TH1_title,TH2_aspect,TH2_title,TH3_aspect,TH3_title
0,3,Tốt nha,quality,5,2,5.0,2,5.0,2,5.0
1,8,"Chất quần xấu, nilon thô bí",quality,1,2,2.0,2,1.0,2,1.0
2,25,"Dùng thích lắm mn ạ , ko hề có mùi , an toàn c...","ship, quality",5,2,5.0,2,5.0,2,5.0
3,10,"sản phẩm đẹp, chất liệu khá tốt đi vào cảm giá...","quality, price",5,2,5.0,2,5.0,2,5.0
4,5,"Nồi nhỏ gọn , áp suất nồi rất tốt , sp đúng ch...","ship, quality, cskh",5,"1,24",5.0,"1,2,4",5.0,"1,24",5.0


In [5]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 597 entries, 0 to 596
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   type        597 non-null    int64  
 1   content     597 non-null    object 
 2   aspect      597 non-null    object 
 3   title       597 non-null    int64  
 4   TH1_aspect  597 non-null    object 
 5   TH1_title   597 non-null    float64
 6   TH2_aspect  597 non-null    object 
 7   TH2_title   597 non-null    float64
 8   TH3_aspect  597 non-null    object 
 9   TH3_title   597 non-null    float64
dtypes: float64(3), int64(2), object(5)
memory usage: 46.8+ KB


In [7]:

# Định nghĩa các từ khóa cho từng khía cạnh
VAN_CHUYEN_KEYWORDS = [
    'giao hàng', 'ship', 'shipper', 'vận chuyển', 'giao', 'nhận hàng', 'đóng gói',
    'thời gian giao', 'giao nhanh', 'giao chậm', 'phí ship', 'thần tốc', 'gói hàng',
    'vận tải', 'giao dịch', 'đơn vị vận chuyển', 'đơn hàng', 'giao đúng hẹn', 'giao trễ',
    'đóng thùng', 'bọc kĩ', 'bọc cẩn thận', 'bưu điện', 'bưu tá', 'giao hỏng', 'giao thiếu',
    'giao đủ', 'giao đúng', 'giao sai', 'vận chuyển nhanh', 'vận chuyển chậm', 'freeship',
    'phí giao hàng', 'giao tận nơi', 'giao tận nhà', 'giao tận tay', 'giao hàng nhanh chóng'
]

CHAT_LUONG_KEYWORDS = [
    'chất lượng', 'bền', 'hỏng', 'tốt', 'xấu', 'đểu', 'kém', 'chắc chắn',
    'dễ vỡ', 'hư', 'dùng tốt', 'sản phẩm tốt', 'hoạt động', 'dùng', 'chất lượng kém', 'nội dung',
    'hay', 'đẹp', 'xịn', 'độc', 'lạ', 'đẹp trai', 'đẹp gái', 'mới', 'cũ', 
    'hàng chính hãng', 'hàng giả', 'hàng nhái', 'tuyệt vời', 'chất liệu', 'vải', 'da', 'nhựa',
    'kim loại', 'gỗ', 'sắt', 'thép', 'nhôm', 'cao su', 'vải dù', 'vải cotton', 'vải len',
    'vải thô', 'vải mịn', 'bền bỉ', 'dễ hỏng', 'dễ hư', 'dễ bể', 'dễ gãy', 'dễ rách',
    'dễ phai màu', 'phai màu', 'bạc màu', 'ố vàng', 'ố màu', 'đổi màu', 'đẹp mắt',
    'xấu xí', 'thô kệch', 'tinh tế', 'tinh xảo', 'thô sơ', 'chất lượng cao', 'cao cấp',
    'sang trọng', 'đẳng cấp', 'đồ rẻ tiền', 'hàng kém chất lượng', 'hàng tốt', 'hàng xịn',
    'hàng đẹp', 'hàng chất', 'hàng chuẩn', 'hàng đúng mô tả', 'đúng như mô tả', 'như hình',
    'giống hình', 'khác hình', 'không giống hình', 'không như mô tả', 'khác mô tả'
]

GIA_CA_KEYWORDS = [
    'giá', 'đắt', 'rẻ', 'khuyến mãi', 'hợp lý', 'giá cả', 'giá trị', 'tiền',
    'đáng tiền', 'không đáng tiền', 'sale', 'giảm giá', 'rẻ hơn', 'đắt hơn',
    'giá tốt', 'giá cao', 'giá thấp', 'giá hời', 'giá mềm', 'giá phải chăng',
    'giá cạnh tranh', 'giá sinh viên', 'giá ưu đãi', 'ưu đãi', 'chiết khấu',
    'voucher', 'mã giảm giá', 'coupon', 'flash sale', 'deal sốc', 'deal hời',
    'giá gốc', 'giá niêm yết', 'giá thị trường', 'giá bán lẻ', 'giá bán buôn',
    'giá tham khảo', 'giá đề xuất', 'giá thành', 'chi phí', 'phí', 'phụ phí',
    'đắt đỏ', 'rẻ mạt', 'rẻ bèo', 'hời', 'lời', 'lãi', 'lỗ', 'bù lỗ', 'xả kho',
    'xả hàng', 'thanh lý', 'giá thanh lý', 'giá xả kho', 'giá xả hàng'
]

CSKH_KEYWORDS = [
    'nhân viên', 'tư vấn', 'hỗ trợ', 'chăm sóc', 'thái độ', 'phục vụ',
    'nhiệt tình', 'phản hồi', 'chăm sóc khách hàng', 'khách hàng', 'dịch vụ',
    'tư vấn viên', 'người bán', 'shop', 'cửa hàng', 'đại lý', 'nhà phân phối',
    'thái độ phục vụ', 'thái độ nhân viên', 'thái độ shop', 'thái độ người bán',
    'phản hồi nhanh', 'phản hồi chậm', 'trả lời nhanh', 'trả lời chậm',
    'tư vấn nhiệt tình', 'tư vấn tận tâm', 'tư vấn chu đáo', 'tư vấn tốt',
    'tư vấn kém', 'tư vấn sai', 'tư vấn đúng', 'hỗ trợ tốt', 'hỗ trợ kém',
    'hỗ trợ nhiệt tình', 'hỗ trợ tận tâm', 'hỗ trợ chu đáo', 'dịch vụ tốt',
    'dịch vụ kém', 'dịch vụ chất lượng', 'dịch vụ tệ', 'phục vụ tốt', 'phục vụ kém',
    'phục vụ chu đáo', 'phục vụ tận tâm', 'phục vụ nhiệt tình', 'bảo hành',
    'đổi trả', 'hoàn tiền', 'chính sách bảo hành', 'chính sách đổi trả',
    'chính sách hoàn tiền', 'chăm sóc tốt', 'chăm sóc kém', 'chăm sóc chu đáo'
]

def contains_keyword(text, keywords):
    """
    Kiểm tra xem văn bản có chứa từ khóa nào trong danh sách không.
    """
    if isinstance(text, str):
        text = text.lower()
        for keyword in keywords:
            if keyword in text:
                return True
    return False

def classify_aspect(text):
    """
    Phân loại khía cạnh của đánh giá dựa trên từ khóa.
    """
    aspects = []
    
    if contains_keyword(text, VAN_CHUYEN_KEYWORDS):
        aspects.append('ship')
    
    if contains_keyword(text, CHAT_LUONG_KEYWORDS):
        aspects.append('quality')
    
    if contains_keyword(text, GIA_CA_KEYWORDS):
        aspects.append('price')
    
    if contains_keyword(text, CSKH_KEYWORDS):
        aspects.append('cskh')
    
    if not aspects:
        aspects.append('other')
    
    return ', '.join(aspects)

# Áp dụng phân loại khía cạnh cho cột content
print("Đang gán nhãn khía cạnh dựa trên từ khóa...")
data['aspect'] = data['content'].apply(classify_aspect)
print("Done")







Đang gán nhãn khía cạnh dựa trên từ khóa...
Done


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 597 entries, 0 to 596
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   type        597 non-null    int64  
 1   content     597 non-null    object 
 2   aspect      597 non-null    object 
 3   title       597 non-null    int64  
 4   TH1_aspect  597 non-null    object 
 5   TH1_title   597 non-null    float64
 6   TH2_aspect  597 non-null    object 
 7   TH2_title   597 non-null    float64
 8   TH3_aspect  597 non-null    object 
 9   TH3_title   597 non-null    float64
dtypes: float64(3), int64(2), object(5)
memory usage: 46.8+ KB


In [9]:
# Tạo ánh xạ từ tên aspect sang số
aspect_mapping = {
    'cskh': 1,
    'quality': 2,
    'price': 3,
    'ship': 4
}

# Hàm chuyển đổi chuỗi aspect thành dạng số
def convert_aspect_to_numeric(aspect_str):
    if pd.isna(aspect_str):
        return None
    
    aspects = [a.strip() for a in aspect_str.split(',')]
    numeric_aspects = [str(aspect_mapping.get(a, 0)) for a in aspects]
    return ','.join(numeric_aspects)

# Áp dụng chuyển đổi cho cột aspect
data['aspect'] = data['aspect'].apply(convert_aspect_to_numeric)

data


,type,content,aspect,title,TH1_aspect,TH1_title,TH2_aspect,TH2_title,TH3_aspect,TH3_title
0,3,Tốt nha,2,5,2,5.0,2,5.0,2,5.0
1,8,"Chất quần xấu, nilon thô bí",2,1,2,2.0,2,1.0,2,1.0
2,25,"Dùng thích lắm mn ạ , ko hề có mùi , an toàn c...",2,5,2,5.0,2,5.0,2,5.0
3,10,"sản phẩm đẹp, chất liệu khá tốt đi vào cảm giá...","2,3",5,2,5.0,2,5.0,2,5.0
4,5,"Nồi nhỏ gọn , áp suất nồi rất tốt , sp đúng ch...","4,2,3,1",5,"1,24",5.0,"1,2,4",5.0,"1,24",5.0
...,...,...,...,...,...,...,...,...,...,...
592,3,"Tốt, bền, giá hợp lý","2,3",5,"2,3",5.0,"2,3",5.0,"2,3",5.0
593,4,"Tai nghe tốt, nghe êm tai",2,5,2,5.0,2,5.0,2,5.0
594,10,"chất liệu thô cứng, rất nóng, đàn hồi kém, vớ ...","2,3",1,2,3.0,2,2.0,2,2.0
595,17,"Hình ảnh cũng ổn , xem rõ",2,4,2,4.0,2,5.0,2,5.0


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 597 entries, 0 to 596
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   type        597 non-null    int64  
 1   content     597 non-null    object 
 2   aspect      597 non-null    object 
 3   title       597 non-null    int64  
 4   TH1_aspect  597 non-null    object 
 5   TH1_title   597 non-null    float64
 6   TH2_aspect  597 non-null    object 
 7   TH2_title   597 non-null    float64
 8   TH3_aspect  597 non-null    object 
 9   TH3_title   597 non-null    float64
dtypes: float64(3), int64(2), object(5)
memory usage: 46.8+ KB


In [12]:
data['type'] = data['type'].astype(int)


# Chuyển đổi các cột 'TH1_title', 'TH2_title', 'TH3_title' sang kiểu float
title_columns = ['TH1_title', 'TH2_title', 'TH3_title']
for col in title_columns:
    data[col] = data[col].astype(int)

# Kiểm tra kiểu dữ liệu sau khi chuyển đổi
print("Kiểm tra kiểu dữ liệu sau khi chuyển đổi:")
print(data.dtypes.head())

# Hiển thị một số dòng để xác nhận
print("\nDữ liệu sau khi chuyển đổi:")
print(data[['type', 'title', 'TH1_title', 'TH2_title', 'TH3_title']].head())


Kiểm tra kiểu dữ liệu sau khi chuyển đổi:
type           int64
content       object
aspect        object
title          int64
TH1_aspect    object
dtype: object

Dữ liệu sau khi chuyển đổi:
   type  title  TH1_title  TH2_title  TH3_title
0     3      5          5          5          5
1     8      1          2          1          1
2    25      5          5          5          5
3    10      5          5          5          5
4     5      5          5          5          5


In [14]:

def convert_aspect_to_list(aspect_str):
    # Nếu đã là list rồi thì trả về luôn
    if isinstance(aspect_str, list):
        return aspect_str
        
    if pd.isna(aspect_str):
        return []
    
    # Tách chuỗi thành các phần
    parts = aspect_str.split(',')
    
    # Danh sách để lưu các số đã xử lý
    result = []
    
    # Xử lý từng phần
    for part in parts:
        part = part.strip()
        # Nếu phần có nhiều hơn 1 chữ số, tách thành các chữ số riêng lẻ
        if len(part) > 1:
            for digit in part:
                if digit.isdigit():
                    result.append(int(digit))
        else:
            if part.isdigit():
                result.append(int(part))
    
    return result


# Áp dụng chuyển đổi cho tất cả các cột aspect
aspect_columns = ['aspect', 'TH1_aspect', 'TH2_aspect', 'TH3_aspect']

for col in aspect_columns:
    # Chuyển đổi cột từ chuỗi sang danh sách số nguyên
    data[col] = data[col].apply(convert_aspect_to_list)

# Kiểm tra kết quả cho tất cả các cột aspect
print("Kiểm tra kết quả chuyển đổi cho tất cả các cột aspect:")
for col in aspect_columns:
    print(f"\nCột {col}:")
    print(data[[col]].head(3))

# Kiểm tra kiểu dữ liệu của các phần tử trong các cột aspect



Kiểm tra kết quả chuyển đổi cho tất cả các cột aspect:

Cột aspect:
  aspect
0    [2]
1    [2]
2    [2]

Cột TH1_aspect:
  TH1_aspect
0        [2]
1        [2]
2        [2]

Cột TH2_aspect:
  TH2_aspect
0        [2]
1        [2]
2        [2]

Cột TH3_aspect:
  TH3_aspect
0        [2]
1        [2]
2        [2]


In [16]:
data

,type,content,aspect,title,TH1_aspect,TH1_title,TH2_aspect,TH2_title,TH3_aspect,TH3_title
0,3,Tốt nha,[2],5,[2],5,[2],5,[2],5
1,8,"Chất quần xấu, nilon thô bí",[2],1,[2],2,[2],1,[2],1
2,25,"Dùng thích lắm mn ạ , ko hề có mùi , an toàn c...",[2],5,[2],5,[2],5,[2],5
3,10,"sản phẩm đẹp, chất liệu khá tốt đi vào cảm giá...","[2, 3]",5,[2],5,[2],5,[2],5
4,5,"Nồi nhỏ gọn , áp suất nồi rất tốt , sp đúng ch...","[4, 2, 3, 1]",5,"[1, 2, 4]",5,"[1, 2, 4]",5,"[1, 2, 4]",5
...,...,...,...,...,...,...,...,...,...,...
592,3,"Tốt, bền, giá hợp lý","[2, 3]",5,"[2, 3]",5,"[2, 3]",5,"[2, 3]",5
593,4,"Tai nghe tốt, nghe êm tai",[2],5,[2],5,[2],5,[2],5
594,10,"chất liệu thô cứng, rất nóng, đàn hồi kém, vớ ...","[2, 3]",1,[2],3,[2],2,[2],2
595,17,"Hình ảnh cũng ổn , xem rõ",[2],4,[2],4,[2],5,[2],5


In [29]:
# Đánh giá độ đồng thuận IAA (Inter-Annotator Agreement)
import numpy as np
from sklearn.metrics import cohen_kappa_score
import krippendorff

print("Đánh giá độ đồng thuận giữa các người chú thích (IAA):")

# ----- Đánh giá cho cột aspect (multi-label) -----
print("\n=== Đánh giá IAA cho cột aspect ===")

# Chuyển đổi danh sách aspect thành tập hợp để so sánh
aspect_sets = {
    'aspect': data['aspect'].apply(set),
    'TH1_aspect': data['TH1_aspect'].apply(set),
    'TH2_aspect': data['TH2_aspect'].apply(set),
    'TH3_aspect': data['TH3_aspect'].apply(set)
}

# Tính Jaccard similarity giữa các cặp người chú thích
def jaccard_similarity(set1, set2):
    if not set1 and not set2:  # Nếu cả hai tập rỗng
        return 1.0
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union > 0 else 0.0

# Tính trung bình Jaccard similarity cho tất cả các cặp
pairs = [
    ('aspect', 'TH1_aspect'),
    ('aspect', 'TH2_aspect'),
    ('aspect', 'TH3_aspect'),
    ('TH1_aspect', 'TH2_aspect'),
    ('TH1_aspect', 'TH3_aspect'),
    ('TH2_aspect', 'TH3_aspect')
]

print("Jaccard similarity giữa các cặp người chú thích:")
for pair in pairs:
    col1, col2 = pair
    similarities = [jaccard_similarity(aspect_sets[col1][i], aspect_sets[col2][i]) 
                   for i in range(len(data))]
    avg_similarity = np.mean(similarities)
    print(f"{col1} vs {col2}: {avg_similarity:.4f}")

# Tính Krippendorff's Alpha cho aspect
# Chuẩn bị dữ liệu cho Krippendorff's Alpha
# Chuyển đổi mỗi aspect thành chuỗi để có thể so sánh
aspect_data = np.array([
    data['aspect'].apply(lambda x: ','.join(map(str, sorted(x)))),
    data['TH1_aspect'].apply(lambda x: ','.join(map(str, sorted(x)))),
    data['TH2_aspect'].apply(lambda x: ','.join(map(str, sorted(x)))),
    data['TH3_aspect'].apply(lambda x: ','.join(map(str, sorted(x))))
]).T

# Tính Krippendorff's Alpha
try:
    alpha = krippendorff.alpha(reliability_data=aspect_data, level_of_measurement='nominal')
    print(f"\nKrippendorff's Alpha cho aspect: {alpha:.4f}")
    
    if alpha < 0.2:
        print("Độ đồng thuận rất thấp")
    elif alpha < 0.4:
        print("Độ đồng thuận thấp")
    elif alpha < 0.6:
        print("Độ đồng thuận trung bình")
    elif alpha < 0.8:
        print("Độ đồng thuận cao")
    else:
        print("Độ đồng thuận rất cao")
except Exception as e:
    print(f"Không thể tính Krippendorff's Alpha: {e}")

# ----- Đánh giá cho cột title (single-label) -----
print("\n=== Đánh giá IAA cho cột title ===")

# Chuyển đổi các cột title thành số nguyên
title_columns = ['title', 'TH1_title', 'TH2_title', 'TH3_title']
for col in title_columns:
    data[col] = data[col].astype(float).astype(int)

# Tính Cohen's Kappa cho từng cặp người chú thích
print("Cohen's Kappa giữa các cặp người chú thích:")
title_pairs = [
    ('title', 'TH1_title'),
    ('title', 'TH2_title'),
    ('title', 'TH3_title'),
    ('TH1_title', 'TH2_title'),
    ('TH1_title', 'TH3_title'),
    ('TH2_title', 'TH3_title')
]

for pair in title_pairs:
    col1, col2 = pair
    kappa = cohen_kappa_score(data[col1], data[col2])
    print(f"{col1} vs {col2}: {kappa:.4f}")

# Tính Fleiss' Kappa cho title
# Chuẩn bị dữ liệu cho Fleiss' Kappa
def fleiss_kappa(ratings, n_categories):
    """
    Tính Fleiss' Kappa cho nhiều người chú thích
    
    Parameters:
    ratings: Ma trận với mỗi hàng là một mục và mỗi cột là một người chú thích
    n_categories: Số lượng danh mục
    
    Returns:
    kappa: Giá trị Fleiss' Kappa
    """
    n_items = ratings.shape[0]  # Số lượng mục đánh giá
    n_raters = ratings.shape[1]  # Số lượng người chú thích
    
    # Đối với aspect, cần xử lý dữ liệu dạng list
    # Đảm bảo rằng [4,1] được coi là giống [1,4]
    # Chuyển đổi mỗi danh sách thành tập hợp (set) để so sánh không phụ thuộc thứ tự
    if isinstance(ratings[0,0], list):
        # Tạo ma trận mới để lưu trữ các tập hợp
        set_ratings = np.empty((n_items, n_raters), dtype=object)
        for i in range(n_items):
            for j in range(n_raters):
                if isinstance(ratings[i,j], list):
                    set_ratings[i,j] = frozenset(ratings[i,j])
                else:
                    set_ratings[i,j] = frozenset([ratings[i,j]])
        ratings = set_ratings
    
    # Tính số lượng người chú thích cho mỗi danh mục và mỗi mục
    count_matrix = np.zeros((n_items, n_categories))
    
    for i in range(n_items):
        for j in range(n_raters):
            category = ratings[i, j] - 1  # Điều chỉnh để bắt đầu từ 0
            if 0 <= category < n_categories:
                count_matrix[i, category] += 1
    
    # Tính P_i (tỷ lệ đồng thuận cho mỗi mục)
    p_i = np.sum(count_matrix * (count_matrix - 1), axis=1) / (n_raters * (n_raters - 1))
    P_bar = np.mean(p_i)
    
    # Tính P_e (tỷ lệ đồng thuận ngẫu nhiên)
    p_j = np.sum(count_matrix, axis=0) / (n_items * n_raters)
    P_e = np.sum(p_j ** 2)
    
    # Tính Fleiss' Kappa
    kappa = (P_bar - P_e) / (1 - P_e)
    
    return kappa

# Chuẩn bị dữ liệu title cho Fleiss' Kappa
title_data = data[title_columns].values
n_categories = 5  # Giả sử có 5 danh mục (1-5)

try:
    fleiss_k = fleiss_kappa(title_data, n_categories)
    print(f"\nFleiss' Kappa cho title: {fleiss_k:.4f}")
    
    if fleiss_k < 0.2:
        print("Độ đồng thuận rất thấp")
    elif fleiss_k < 0.4:
        print("Độ đồng thuận thấp")
    elif fleiss_k < 0.6:
        print("Độ đồng thuận trung bình")
    elif fleiss_k < 0.8:
        print("Độ đồng thuận cao")
    else:
        print("Độ đồng thuận rất cao")
except Exception as e:
    print(f"Không thể tính Fleiss' Kappa: {e}")

# Tính tỷ lệ đồng thuận tuyệt đối (Exact Agreement)

def exact_agreement_rate_fixed(data, columns):
    """Tính tỷ lệ các mục mà tất cả người chú thích đều đồng ý hoàn toàn"""
    total_items = len(data)
    agreement_count = 0
    
    for _, row in data.iterrows():
        values = [row[col] for col in columns]
        
        # Chuyển đổi các giá trị list thành tuple (hashable) hoặc so sánh trực tiếp
        if isinstance(values[0], list):
            # Nếu giá trị là list, chuyển thành string để so sánh
            str_values = [str(sorted(val)) for val in values]
            if len(set(str_values)) == 1:  # Tất cả giá trị giống nhau
                agreement_count += 1
        else:
            # Nếu không phải list, so sánh bình thường
            if len(set(values)) == 1:  # Tất cả giá trị giống nhau
                agreement_count += 1
    
    return agreement_count / total_items

# Sử dụng hàm đã sửa để tính tỷ lệ đồng thuận
aspect_agreement_rate = exact_agreement_rate_fixed(data, aspect_columns)
title_agreement_rate = exact_agreement_rate_fixed(data, title_columns)

print(f"\nTỷ lệ đồng thuận tuyệt đối cho aspect: {aspect_agreement_rate:.4f}")
print(f"Tỷ lệ đồng thuận tuyệt đối cho title: {title_agreement_rate:.4f}")


Đánh giá độ đồng thuận giữa các người chú thích (IAA):

=== Đánh giá IAA cho cột aspect ===
Jaccard similarity giữa các cặp người chú thích:
aspect vs TH1_aspect: 0.8221
aspect vs TH2_aspect: 0.8223
aspect vs TH3_aspect: 0.8221
TH1_aspect vs TH2_aspect: 0.9373
TH1_aspect vs TH3_aspect: 1.0000
TH2_aspect vs TH3_aspect: 0.9373
Không thể tính Krippendorff's Alpha: Don't know how to construct value domain for dtype kind O.

=== Đánh giá IAA cho cột title ===
Cohen's Kappa giữa các cặp người chú thích:
title vs TH1_title: 0.5223
title vs TH2_title: 0.5247
title vs TH3_title: 0.5247
TH1_title vs TH2_title: 0.6440
TH1_title vs TH3_title: 0.6440
TH2_title vs TH3_title: 1.0000

Fleiss' Kappa cho title: 0.6451
Độ đồng thuận cao

Tỷ lệ đồng thuận tuyệt đối cho aspect: 0.6131
Tỷ lệ đồng thuận tuyệt đối cho title: 0.7672


In [31]:
# Đo lường độ đồng thuận giữa hệ thống và con người

print("\n" + "="*50)
print("ĐO LƯỜNG ĐỘ ĐỒNG THUẬN GIỮA HỆ THỐNG VÀ CON NGƯỜI")
print("="*50)

from sklearn.metrics import cohen_kappa_score
import numpy as np

# Tạo nhãn tiêu chuẩn vàng từ đa số người đánh giá
def create_gold_standard(data, columns):
    """Tạo nhãn tiêu chuẩn vàng dựa trên đa số người đánh giá"""
    gold_standard = []
    
    for _, row in data.iterrows():
        values = [row[col] for col in columns]
        
        if isinstance(values[0], list):
            # Đếm tần suất xuất hiện của mỗi phần tử trong tất cả các danh sách
            all_elements = []
            for val_list in values:
                all_elements.extend(val_list)
            
            # Đếm tần suất
            element_counts = {}
            for elem in all_elements:
                if elem in element_counts:
                    element_counts[elem] += 1
                else:
                    element_counts[elem] = 1
            
            # Chọn các phần tử xuất hiện ở ít nhất 2/3 người đánh giá
            threshold = len(columns) / 2  # Ngưỡng đa số (hơn 1/2)
            gold_elements = [elem for elem, count in element_counts.items() if count > threshold]
            
            # Nếu không có phần tử nào vượt ngưỡng, lấy phần tử xuất hiện nhiều nhất
            if not gold_elements and element_counts:
                max_count = max(element_counts.values())
                gold_elements = [elem for elem, count in element_counts.items() if count == max_count]
            
            gold_standard.append(sorted(gold_elements))
        else:
            # Đối với giá trị đơn, lấy giá trị xuất hiện nhiều nhất
            value_counts = {}
            for val in values:
                if val in value_counts:
                    value_counts[val] += 1
                else:
                    value_counts[val] = 1
            
            max_count = max(value_counts.values())
            most_common = [val for val, count in value_counts.items() if count == max_count]
            gold_standard.append(most_common[0])  # Lấy giá trị đầu tiên nếu có nhiều giá trị cùng tần suất
    
    return gold_standard

# Tính Cohen's Kappa cho nhãn đơn (title)
def calculate_cohen_kappa_for_title(data, human_columns, system_column):
    gold_standard = create_gold_standard(data, human_columns)
    system_labels = data[system_column].tolist()
    
    return cohen_kappa_score(gold_standard, system_labels)

# Tính Cohen's Kappa cho nhãn đa nhãn (aspect)
def calculate_cohen_kappa_for_aspects(data, human_columns, system_column):
    gold_standard = create_gold_standard(data, human_columns)
    system_labels = data[system_column].tolist()
    
    # Chuyển đổi danh sách aspect thành chuỗi để so sánh
    gold_standard_str = [str(sorted(aspects)) for aspects in gold_standard]
    system_labels_str = [str(sorted(aspects)) for aspects in system_labels]
    
    return cohen_kappa_score(gold_standard_str, system_labels_str)

# Tính F1-score cho từng aspect riêng lẻ
def calculate_aspect_f1_scores(data, human_columns, system_column):
    gold_standard = create_gold_standard(data, human_columns)
    system_labels = data[system_column].tolist()
    
    # Xác định tất cả các aspect có thể có
    all_aspects = set()
    for aspects in gold_standard + system_labels:
        all_aspects.update(aspects)
    
    results = {}
    for aspect in all_aspects:
        tp = 0  # True Positive
        fp = 0  # False Positive
        fn = 0  # False Negative
        
        for gold, system in zip(gold_standard, system_labels):
            if aspect in gold and aspect in system:
                tp += 1
            elif aspect not in gold and aspect in system:
                fp += 1
            elif aspect in gold and aspect not in system:
                fn += 1
        
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        
        results[aspect] = {
            'precision': precision,
            'recall': recall,
            'f1': f1
        }
    
    return results

# Thực hiện đánh giá
try:
    # Đánh giá cho title
    title_kappa = calculate_cohen_kappa_for_title(data, title_columns, 'title')
    print(f"\nCohen's Kappa cho title (Hệ thống vs Con người): {title_kappa:.4f}")
    
    if title_kappa < 0.2:
        print("Độ đồng thuận rất thấp")
    elif title_kappa < 0.4:
        print("Độ đồng thuận thấp")
    elif title_kappa < 0.6:
        print("Độ đồng thuận trung bình")
    elif title_kappa < 0.8:
        print("Độ đồng thuận cao")
    else:
        print("Độ đồng thuận rất cao")
    
    # Đánh giá cho aspect
    aspect_kappa = calculate_cohen_kappa_for_aspects(data, aspect_columns, 'aspect')
    print(f"\nCohen's Kappa cho aspect (Hệ thống vs Con người): {aspect_kappa:.4f}")
    
    if aspect_kappa < 0.2:
        print("Độ đồng thuận rất thấp")
    elif aspect_kappa < 0.4:
        print("Độ đồng thuận thấp")
    elif aspect_kappa < 0.6:
        print("Độ đồng thuận trung bình")
    elif aspect_kappa < 0.8:
        print("Độ đồng thuận cao")
    else:
        print("Độ đồng thuận rất cao")
    
    # Đánh giá chi tiết cho từng aspect
    print("\nĐánh giá chi tiết cho từng aspect:")
    aspect_f1_scores = calculate_aspect_f1_scores(data, aspect_columns, 'aspect')
    
    # Ánh xạ mã aspect sang tên
    aspect_names = {
        0: 'other',
        1: 'cskh',
        2: 'quality',
        3: 'price',
        4: 'ship'
    }
    
    print("\n{:<10} {:<12} {:<12} {:<12}".format("Aspect", "Precision", "Recall", "F1-Score"))
    print("-" * 50)
    
    for aspect, scores in aspect_f1_scores.items():
        aspect_name = aspect_names.get(aspect, str(aspect))
        print("{:<10} {:<12.4f} {:<12.4f} {:<12.4f}".format(
            aspect_name, 
            scores['precision'], 
            scores['recall'], 
            scores['f1']
        ))
    
    # Tính trung bình
    avg_precision = np.mean([scores['precision'] for scores in aspect_f1_scores.values()])
    avg_recall = np.mean([scores['recall'] for scores in aspect_f1_scores.values()])
    avg_f1 = np.mean([scores['f1'] for scores in aspect_f1_scores.values()])
    
    print("-" * 50)
    print("{:<10} {:<12.4f} {:<12.4f} {:<12.4f}".format(
        "Trung bình", avg_precision, avg_recall, avg_f1
    ))
    
except Exception as e:
    print(f"Lỗi khi đánh giá độ đồng thuận: {e}")



ĐO LƯỜNG ĐỘ ĐỒNG THUẬN GIỮA HỆ THỐNG VÀ CON NGƯỜI

Cohen's Kappa cho title (Hệ thống vs Con người): 0.6431
Độ đồng thuận cao

Cohen's Kappa cho aspect (Hệ thống vs Con người): 0.5696
Độ đồng thuận trung bình

Đánh giá chi tiết cho từng aspect:

Aspect     Precision    Recall       F1-Score    
--------------------------------------------------
cskh       0.3846       0.8163       0.5229      
quality    0.9371       0.8936       0.9149      
price      0.6159       0.9341       0.7424      
ship       0.9452       0.9452       0.9452      
--------------------------------------------------
Trung bình 0.7207       0.8973       0.7813      
